In [1]:
#libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [2]:
import os

from google.colab import drive
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/Data_Science/Data_Frames/Bank_Term'

Mounted at /content/drive


In [3]:
data1 = pd.read_csv('/content/drive/MyDrive/Data_Science/Data_Frames/Bank_Term/train.csv')
data2 = pd.read_csv('/content/drive/MyDrive/Data_Science/Data_Frames/Bank_Term/test.csv')

train_df = data1.copy()
test_df = data2.copy()

print("First 5 rows of train_df: \n", train_df.head(5))
print("First 5 rows of test_df: \n", test_df.head(5))

First 5 rows of train_df: 
    id  age          job  marital  education default  balance housing loan  \
0   0   42   technician  married  secondary      no        7      no   no   
1   1   38  blue-collar  married  secondary      no      514      no   no   
2   2   36  blue-collar  married  secondary      no      602     yes   no   
3   3   27      student   single  secondary      no       34     yes   no   
4   4   26   technician  married  secondary      no      889     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0  cellular   25   aug       117         3     -1         0  unknown  0  
1   unknown   18   jun       185         1     -1         0  unknown  0  
2   unknown   14   may       111         2     -1         0  unknown  0  
3   unknown   28   may        10         2     -1         0  unknown  0  
4  cellular    3   feb       902         1     -1         0  unknown  1  
First 5 rows of test_df: 
        id  age            job  marital

In [4]:
train_df['was_contacted_previous'] = train_df['pdays'].apply(lambda x: 1 if x > 0 else 0)
test_df['was_contacted_previous'] = test_df['pdays'].apply(lambda x: 1 if x > 0 else 0)

In [5]:
train_df.loc[train_df['pdays'] == -1, 'pdays'] = 99999
test_df.loc[test_df['pdays'] == -1, 'pdays'] = 99999

In [6]:
test_id_placeholder = test_df['id']

In [7]:
cat_cols=train_df.select_dtypes(include=['object']).columns
num_cols=train_df.select_dtypes(include=['int']).columns

print(f'Total Categorical Columns {len(cat_cols)}')
print(f'Total Numerical Columns {len(num_cols)}')

Total Categorical Columns 9
Total Numerical Columns 10


In [8]:
from sklearn.model_selection import train_test_split

X = train_df.drop(columns=['id', 'y'])
y = train_df['y']

test_df = test_df.drop(columns='id')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=42)

In [9]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 16.2 MB/s eta 0:00:00


In [10]:
cat_cols = X.select_dtypes(include=['object']).columns.tolist()
cat_cols.append('was_contacted_previous')

In [11]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.5 MB/s eta 0:00:00


In [12]:
from sklearn.model_selection import StratifiedKFold,train_test_split
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score
import optuna

In [13]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 500, 2500),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.1),
        'depth': trial.suggest_int('depth', 4, 10),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),
        'random_state': 42,
        'eval_metric': 'AUC',
        'verbose': 0,
        'early_stopping_rounds': 50,
        'task_type': 'GPU',
        'devices': '0'
    }
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    train_pool = Pool(X_train, y_train, cat_features=cat_cols)
    test_pool = Pool(X_test, y_test, cat_features=cat_cols)
    model = CatBoostClassifier(**params)
    model.fit(train_pool, eval_set=test_pool)
    y_prob = model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_prob)

    return roc_auc

In [14]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5, show_progress_bar=True)
best_params = study.best_trial.params
print("\nBest Hyperparameters from Optuna:")
print(best_params)

[I 2025-08-19 13:42:57,903] A new study created in memory with name: no-name-e0a36dc0-c5d7-4594-8017-4c229e23b2f0


  0%|          | 0/5 [00:00<?, ?it/s]

[W 2025-08-19 13:43:01,699] Trial 0 failed with parameters: {'iterations': 1762, 'learning_rate': 0.05562242328263331, 'depth': 4, 'l2_leaf_reg': 1.2151431457711301} because of the following error: CatBoostError('catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-43304284.py", line 18, in objective
    model.fit(train_pool, eval_set=test_pool)
  File "/usr/local/lib/python3.11/dist-packages/catboost/core.py", line 5245, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/usr/local/lib/python3.11/dist-packages/catboost/core.py", line 2410, in _fit
    self._train(
  File "/usr/local/li

CatBoostError: catboost/cuda/cuda_lib/cuda_base.h:281: CUDA error 35: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
best_params['random_state'] = 42
best_params['eval_metric'] = 'AUC'
best_params['verbose'] = 100
best_params['early_stopping_rounds'] = 50

In [ ]:
NFOLDS = 5
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(test_df))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    print(f"\n--- Starting Fold {fold_ + 1}/{NFOLDS} ---")

    trn_data = X.iloc[trn_idx]
    val_data = X.iloc[val_idx]
    trn_y = y.iloc[trn_idx]
    val_y = y.iloc[val_idx]

    model = CatBoostClassifier(**best_params)

    model.fit(trn_data, trn_y, cat_features=cat_cols,
              eval_set=(val_data, val_y), use_best_model=True)

    oof_preds[val_idx] = model.predict_proba(val_data)[:, 1]
    test_preds += model.predict_proba(test_df)[:, 1] / NFOLDS